In [103]:
def secret_santa(liste_participants, config):
    resultats = []
    offrants_restant = liste_participants.copy()
    recevants_restant = liste_participants.copy()
    exclusions = config['exclusions'].copy()
    obligations = config['obligations'].copy()

    # on traite d'abord les obligations
    for offrant, recevant in obligations:
        resultats.append((offrant, recevant))
        offrants_restant.pop(offrants_restant.index(offrant))
        recevants_restant.pop(recevants_restant.index(recevant))
        
    # on parcours au hasard la liste des participants
    for i_donnant_a_traiter in np.random.permutation(len(offrants_restant)):
        donnant_a_traiter = offrants_restant[i_donnant_a_traiter]
        # on tire au hasard les recevants restants
        for i_recevant_possible in np.random.permutation(len(recevants_restant)):
            # vérification des exclusions
            if (donnant_a_traiter, recevants_restant[i_recevant_possible]) not in exclusions and donnant_a_traiter != recevants_restant[i_recevant_possible]:
                resultats.append((donnant_a_traiter, recevants_restant[i_recevant_possible]))
                recevants_restant.pop(i_recevant_possible)
                break
            else:
                continue
    return(resultats)


In [106]:
liste_participants = ['Papa', 'Maman', 'Didier', 'Anne', 'Cécile', 'Mamy', 'Bruno']
config = {
    'exclusions':[
        ('Papa', 'Maman'),
        ('Maman', 'Papa'),
        ('Cecile', 'Bruno'),
        ('Bruno', 'Cecile'),
        ('Anne', 'Didier'),
        ('Didier', 'Anne')],
    'obligations':[
        ('Bruno', 'Papa'),
        ('Didier', 'Maman')]
}
secret_santa_results = secret_santa(liste_participants, config)
pd.DataFrame(secret_santa_results, columns = ['offrant', 'recevant'])

,offrant,recevant
0,Bruno,Papa
1,Didier,Maman
2,Cécile,Didier
3,Anne,Bruno
4,Mamy,Cécile
5,Papa,Anne
6,Maman,Mamy


In [107]:
liste_participants = ['TERRIER Viktor', 'FINET Sebastien', 'GIROD Marie', 'COGEN Florent', 'DUMAS Jonathan', 'LITTLE Emily', 'DUSSARTRE Virginie', 'HAMDANE Ibtissam', 
                      'ARULPRAGASAM Steven', 'MOTHE Axel', 'BUSTARRET Quentin', 'LEMETAYER Bruno']
config = {
    'exclusions':[],
    'obligations':[]}
secret_santa_results = secret_santa(liste_participants, config)
pd.DataFrame(secret_santa_results, columns = ['offrant', 'recevant'])

,offrant,recevant
0,LITTLE Emily,ARULPRAGASAM Steven
1,COGEN Florent,TERRIER Viktor
2,DUMAS Jonathan,DUSSARTRE Virginie
3,LEMETAYER Bruno,FINET Sebastien
4,HAMDANE Ibtissam,DUMAS Jonathan
5,TERRIER Viktor,LEMETAYER Bruno
6,FINET Sebastien,COGEN Florent
7,ARULPRAGASAM Steven,HAMDANE Ibtissam
8,MOTHE Axel,LITTLE Emily
9,GIROD Marie,MOTHE Axel
